## Import packages

First, let's import needed modules and, random seed (we'll use it if needed) and create some auxiliary functions.

In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
from itertools import islice, chain
from nltk import ngrams
from tqdm.notebook import tqdm
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Tuple, Iterator

SEED=42

In [2]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

## Data preparation

I'll be using dataset from [Spooky Author Identification](https://www.kaggle.com/c/spooky-author-identification/overview) competition

### Loading the data

In [39]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

### Data Fields

* id - a unique identifier for each sentence
* text - some text written by one of the authors
* author - the author of the sentence (EAP: Edgar Allan Poe, HPL: HP Lovecraft; MWS: Mary Wollstonecraft Shelley)

Let's look at the data

In [40]:
train_df.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


For now, I'm going to look only at column text to look at the ways text representation can be done

### Data Splitting

But nevertheless let's split the data into training and validation sets.  
As soon as we have almost $20 000$ rows in `train_df`, test size will be limited to $10\%$

In [41]:
train, val = train_test_split(train_df, test_size=0.1, random_state=SEED)

## Word embeddings

### Bag of words

#### One-hot vectors 

The simplest way of word representation is **one-hot vectors**. For the i-th word in the vocabulary, the vector has 1 on the i-th dimension and 0 on the rest.   
Let's do this using sklearn's `CountVectorizer` with `binary=True`

In [42]:
count_vect = CountVectorizer(binary=True)
X_train_oh = count_vect.fit_transform(train['text'])
print(f"The size of the train dataset is {X_train_oh.shape}")

The size of the train dataset is (17621, 24069)


By default, we are not limiting the vocabulary of the model and the length of the vector for every sentence will be $24$ $069$ - number of words in our vocab.
Although, it can be done by setting parameter `max_features` to, for example, $10000$. By doing this, vocabulary will be built considering only the top `max_features` ordered by term frequency across the corpus.

In [43]:
X_train_oh

<17621x24069 sparse matrix of type '<class 'numpy.int64'>'
	with 386417 stored elements in Compressed Sparse Row format>

It is also worth to mention, that due to the sparsity of representation (most values in word vectors will be zeros) we can save a lot of memory by only storing the non-zero parts of the feature vectors in memory. `scipy.sparse` matrices are data structures that do exactly this and they are used in `sklearn` package.

In [44]:
17621*24069

424119849

In our case only $386$ $417$ of elements out of $424$ $119$ $849$ are non-zero.

In [45]:
count_vect_lim_vocab = CountVectorizer(binary=True, max_features=10_000)
X_train_oh = count_vect_lim_vocab.fit_transform(train['text'])
X_train_oh.shape

(17621, 10000)

Now, the length of the vector is $10000$.

Let's look at the vector for the arbitrary sentence in our dataset.

In [46]:
sentence = val['text'][6148]

In [47]:
one_hot_vector = count_vect.transform([sentence])
one_hot_vector

<1x24069 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [48]:
one_hot_vector = one_hot_vector.toarray()
one_hot_vector

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

It is a sparse vector with $\text{10 000}$ elements with only $16$ elements that are not equal to zero. 
Let's find out which are they.

In [49]:
indices = np.where(np.any(one_hot_vector!=0, axis=0))[0]
indices

array([  614,   813,  1548,  2128,  5189,  9205, 10643, 12834, 14023,
       14557, 20543, 21197, 21300, 21556, 22713, 23613], dtype=int64)

These are the indices of words which are present in our sentence.  
Now we are going to create index-to-word dictionary to check the result of the work of `CountVectorizer`

In [50]:
index_to_word = {index : word for word, index in count_vect.vocabulary_.items()}
dict(take(10, index_to_word.items()) )

{20160: 'still',
 14787: 'others',
 10869: 'including',
 11721: 'joe',
 10157: 'himself',
 9894: 'have',
 21222: 'theories',
 21548: 'too',
 23652: 'wild',
 813: 'and'}

In [51]:
[(ind, index_to_word[ind], one_hot_vector[0, ind]) for ind in indices]

[(614, 'all', 1),
 (813, 'and', 1),
 (1548, 'available', 1),
 (2128, 'bewildered', 1),
 (5189, 'dazzled', 1),
 (9205, 'gigantic', 1),
 (10643, 'immediately', 1),
 (12834, 'magnitude', 1),
 (14023, 'nature', 1),
 (14557, 'of', 1),
 (20543, 'sum', 1),
 (21197, 'the', 1),
 (21300, 'thought', 1),
 (21556, 'topic', 1),
 (22713, 'upon', 1),
 (23613, 'who', 1)]

Indeed, these are the indices and corresponding words from our sentence

Because we've created the `CountVectorizer` with `binary=True`. The elements are really ones and zeros.  

#### One-hot vectors with counts

A little improvement over that will be using vectorizer with `binary=False`, because this way we will take counts into account.

In [52]:
count_vect = CountVectorizer(binary=False)
X_train_cv = count_vect.fit_transform(train['text'])

one_hot_vector = count_vect.transform([sentence]).toarray()
indices = np.where(np.any(one_hot_vector!=0, axis=0))[0]
index_to_word = {index : word for word, index in count_vect.vocabulary_.items()}

[(ind, index_to_word[ind], one_hot_vector[0, ind]) for ind in indices]

[(614, 'all', 1),
 (813, 'and', 2),
 (1548, 'available', 1),
 (2128, 'bewildered', 1),
 (5189, 'dazzled', 1),
 (9205, 'gigantic', 1),
 (10643, 'immediately', 1),
 (12834, 'magnitude', 1),
 (14023, 'nature', 1),
 (14557, 'of', 1),
 (20543, 'sum', 1),
 (21197, 'the', 4),
 (21300, 'thought', 1),
 (21556, 'topic', 1),
 (22713, 'upon', 1),
 (23613, 'who', 1)]

We can see that now the value for 'the' is 4 and for 'two' it is 2. It doesn't only show that these words are present in the sentence, but also indicate how many times they occur in the sentence.

In [53]:
tokenizer = count_vect.build_tokenizer()
tokenized_sentence = tokenizer(sentence.lower())
tokenized_sentence.count('the')

4

#### N-grams

We can take into account not only words, but collocations using parameter `ngram_range` to preserve some local ordering, because by using unigrams we don't capture even that.  

In [19]:
bigram_count_vect = CountVectorizer(binary=False, ngram_range=(1,2))
X_train_bigram = bigram_count_vect.fit_transform(train['text'])
print(f"The size of the train dataset is {X_train_bigram.shape}")
print(X_train_bigram.count_nonzero)
one_hot_vector = bigram_count_vect.transform([sentence]).toarray()
indices = np.where(np.any(one_hot_vector!=0, axis=0))[0]
index_to_word = {index : word for word, index in bigram_count_vect.vocabulary_.items()}

print([(ind, index_to_word[ind], one_hot_vector[0, ind]) for ind in indices])

The size of the train dataset is (17621, 230440)
<bound method _data_matrix.count_nonzero of <17621x230440 sparse matrix of type '<class 'numpy.int64'>'
	with 813907 stored elements in Compressed Sparse Row format>>
[(4324, 'all', 1), (4855, 'all who', 1), (7667, 'and', 2), (8116, 'and bewildered', 1), (12202, 'and the', 1), (19515, 'available', 1), (25467, 'bewildered', 1), (45745, 'dazzled', 1), (75382, 'gigantic', 1), (93018, 'immediately', 1), (112540, 'magnitude', 1), (112541, 'magnitude and', 1), (124876, 'nature', 1), (124942, 'nature of', 1), (131308, 'of', 1), (134836, 'of the', 1), (181040, 'sum', 1), (186670, 'the', 4), (189413, 'the gigantic', 1), (192991, 'the sum', 1), (193291, 'the topic', 1), (199054, 'thought', 1), (199173, 'thought upon', 1), (203935, 'topic', 1), (209892, 'upon', 1), (210124, 'upon the', 1), (222194, 'who', 1), (222471, 'who thought', 1)]


By including bigrams into calculations the size of the vector increased from $24$ $069$ to $230$ $440$, but we still can limit it using `max_features` parameter, so it is not a big deal.

#### Stopwords

Removing or not removing stopwords is a controversial topic...

#### 3.1.5 Advantages and disadvantages

$"+"$:
1. Fast to train (actually, there is no training done - just counting)

$"-"$:
1. Vector dimensionality is equal to vocabulary size (`max_features`) 
2. Longer documents will have higher average count values than shorter documents, even though they might talk about the same topics
3. **One-hot vectors don't capture meaning**

### Baseline model

To evaluate embeddings we will stick to extrinsic evaluation - evaluation on real task - text classification

In [20]:
count_vect_eval = CountVectorizer(ngram_range=(1,1))
X_train = count_vect_eval.fit_transform(train['text'])
X_val = count_vect_eval.transform(val['text'])

In [21]:
logreg_count = LogisticRegression(solver='sag', max_iter=10000)
logreg_count.fit(X_train, train['author'])

LogisticRegression(max_iter=10000, solver='sag')

In [22]:
pred = logreg_count.predict(X_val)
f1 = f1_score(pred, val['author'], average='macro')
f1

0.8226872991519389

### TF-IDF

#### Understanding

To address second issue of bag of words approach and in attempt of trying to deal with common words (stopwords) without deleting them the next approach can be used.  

Tf means term-frequency while tf–idf means term-frequency times inverse document-frequency:
$$ \large tf-idf(t, d) = tf(t,d) * idf(t)$$  
Term frequency is: 
$$ \large tf(t, d) = \frac{wordCount(t,d)}{length(d)}$$ 
where:
* `wordCount(t, d)` is the number of occurrences of term *t* in the document *d*
* `length(d)` is the number of words in the document *d*  

Dividing the number of occurrences of each word in a document by the total number of words in the document helps to solve the issue mentioned in the first sentence of this block.

Inverse document-frequency is:
$$ \large idf(t, c) = \frac{size(c)}{docCount(t, c)}$$ 
where: 
* `size(c)` is the number of documents in the corpora *c*
* `docCount(t, c)` is the number of documents in corpora *c* containing the term *t*  

Ifd helps to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.

#### Real life form

In real life tf-idf is computed differently:
$$ \large tf(t,d) = \log{ \left(1 + \frac{wordCount(t,d)}{length(d)}\right)}$$ 
with `sublinear_tf=True` for `sklearn.feature_extraction.text.TfidfVectorizer`
$$ \large idf(t, c) = 1 + \log{\left(\frac{1 + size(c)}{1 + docCount(t, c)}\right)}$$ 
with `norm='l2', use_idf=True, smooth_idf=True` for `sklearn.feature_extraction.text.TfidfVectorizer`

The reason to use `log` here is ***TODO***. 

Vector with tf-idf scores for the words in the sentence should be normalized by the Euclidean norm:
$$\large v_{norm} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v_{1}^2 + v_{2}^2 + ... + v_{n}^2}}$$
By doing this each output vector will have l2 unit norm. The cosine similarity between two vectors is their dot product when l2 norm has been applied.

#### Sklearn's implementation

In [23]:
tfidf_vect = TfidfVectorizer()
X_train_tf_idf = tfidf_vect.fit_transform(train['text'])
print(f"The size of the train dataset is {X_train_tf_idf.shape}")

The size of the train dataset is (17621, 24069)


In [24]:
X_train_tf_idf

<17621x24069 sparse matrix of type '<class 'numpy.float64'>'
	with 386417 stored elements in Compressed Sparse Row format>

In [25]:
tfidf_vect.transform([sentence])

<1x24069 sparse matrix of type '<class 'numpy.float64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [26]:
index_to_word = {index : word for word, index in tfidf_vect.vocabulary_.items()}

In [27]:
tfidf_vector = tfidf_vect.transform([sentence])
tfidf_vector

<1x24069 sparse matrix of type '<class 'numpy.float64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [28]:
tfidf_vector = tfidf_vector.toarray()
tfidf_vector

array([[0., 0., 0., ..., 0., 0., 0.]])

In [29]:
indices = np.where(np.any(tfidf_vector!=0, axis=0))[0]
indices

array([  614,   813,  1548,  2128,  5189,  9205, 10643, 12834, 14023,
       14557, 20543, 21197, 21300, 21556, 22713, 23613], dtype=int64)

In [30]:
[(ind, index_to_word[ind], tfidf_vector[0, ind]) for ind in indices]

[(614, 'all', 0.13031453897504672),
 (813, 'and', 0.12214845066990851),
 (1548, 'available', 0.3924242175008267),
 (2128, 'bewildered', 0.3114996887796031),
 (5189, 'dazzled', 0.36544937459375215),
 (9205, 'gigantic', 0.2695648987205094),
 (10643, 'immediately', 0.23647294743920957),
 (12834, 'magnitude', 0.32608143594739347),
 (14023, 'nature', 0.2049931029681656),
 (14557, 'of', 0.059811003957834306),
 (20543, 'sum', 0.3009169879304377),
 (21197, 'the', 0.20393197647953104),
 (21300, 'thought', 0.18711801923161203),
 (21556, 'topic', 0.2973766872628445),
 (22713, 'upon', 0.14506693111726507),
 (23613, 'who', 0.16239686486865726)]

We see that the words that are popular among the sentences in the corpus have a much lower tf-idf score, than less popular ones.
For example, words that have:
* Lowest scores: all, an, of, upon *often occur in the sentences, thus cannot help to discern the sentences between themselves*.
* Highest scores: available, dazzled, magnitude, sum, topic *are probably rarely found in the sentences in the corpus of texts, but are available in this sentence and thus will make a good features*.

#### TF-IDF implementation

I'm going to implement tf-idf from scratch and compare results with sklearn's implementation.  
However, to save time I will use already trained tokenizer from sklearn.  
*Disclaimer: this implementation is not optimized and not tested properly*

In [31]:
tfidf_tokenizer = tfidf_vect.build_tokenizer()
sentence_tokenized = tfidf_tokenizer(sentence.lower())
sentence_tokenized_set = list(set(sentence_tokenized))

I will get the scores for the same sentence 'The gigantic magnitude and the immediately available nature of the sum, dazzled and bewildered all who thought upon the topic.'

In [32]:
def get_tf(tokenized_text, term):
    return tokenized_text.count(term) / len(tokenized_text)

def get_idf(tokenized_corpus, term, tokenizer):
    size = len(tokenized_corpus)
    doc_count = sum([1 if term in text else 0 for text in tokenized_corpus])
    
    return np.log((1 + size) / (1 + doc_count)) + 1

def get_tf_idf(tf, idf):
    return tf * idf

def get_l2_norm(vector):
    return np.sqrt(np.sum(vector**2))

In [33]:
tf_idf_first_sentece = []
tokenized_corpus = [tokenizer(text.lower()) for text in train['text']]

for word in sentence_tokenized_set:
    tf_ = get_tf(sentence_tokenized, word)
    idf_ = get_idf(tokenized_corpus, word, tfidf_tokenizer)
    tf_idf_first_sentece.append(get_tf_idf(tf_, idf_))
    
tf_idf_first_sentece = np.array(tf_idf_first_sentece)
l2_norm = get_l2_norm(tf_idf_first_sentece)
tf_idf_first_sentece = tf_idf_first_sentece / l2_norm

In [34]:
tf_idf_sentence_dict = dict(zip(sentence_tokenized_set, tf_idf_first_sentece))

In [35]:
[(ind, index_to_word[ind], tf_idf_sentence_dict[index_to_word[ind]]) for ind in indices]

[(614, 'all', 0.13031453897504672),
 (813, 'and', 0.12214845066990852),
 (1548, 'available', 0.3924242175008267),
 (2128, 'bewildered', 0.3114996887796031),
 (5189, 'dazzled', 0.36544937459375215),
 (9205, 'gigantic', 0.2695648987205094),
 (10643, 'immediately', 0.23647294743920957),
 (12834, 'magnitude', 0.32608143594739347),
 (14023, 'nature', 0.2049931029681656),
 (14557, 'of', 0.05981100395783431),
 (20543, 'sum', 0.3009169879304377),
 (21197, 'the', 0.20393197647953104),
 (21300, 'thought', 0.18711801923161206),
 (21556, 'topic', 0.2973766872628445),
 (22713, 'upon', 0.1450669311172651),
 (23613, 'who', 0.16239686486865726)]

In the end, we are getting the same results as in sklearn's implementation with default parameters.

#### Main parameters

* `strip_accents`: Remove accents and perform other character normalization during the preprocessing step. 
* `stop_words`: If a string, it is passed to _check_stop_list and the appropriate stop list is returned. ‘english’ is currently the only supported string value.
* `ngram_range`: The lower and upper boundary of the range of n-values for different n-grams to be extracted. All values of n such that min_n <= n <= max_n will be used. For example an ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams.
* `max_df`: When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float in range [0.0, 1.0], the parameter represents a proportion of documents, integer absolute counts.
* `min_df`: When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float in range of [0.0, 1.0], the parameter represents a proportion of documents, integer absolute counts.
* `max_features`: If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
* `binary`: If True, all non-zero term counts are set to 1. This does not mean outputs will have only 0/1 values, only that the tf term in tf-idf is binary. (Set idf and normalization to False to get 0/1 outputs).
* `norm`: Each output row will have unit norm, either:
    * ‘l2’: Sum of squares of vector elements is 1. The cosine similarity between two vectors is their dot product when l2 norm has been applied.
    * ‘l1’: Sum of absolute values of vector elements is 1. See preprocessing.normalize.
* `use_idf`: Enable inverse-document-frequency reweighting. If False, idf(t) = 1.
* `smooth_idf`: Smooth idf weights by adding one to document frequencies, as if an extra document was seen containing every term in the collection exactly once. Prevents zero divisions.
* `sublinear_tf`: Apply sublinear tf scaling, i.e. replace tf with 1 + log(tf).

Hyperparameters to use for optimization:
* max_df
* min_df
* max_features
* ngram_range

#### N-grams

In [36]:
bigram_tfidf_vect = TfidfVectorizer(ngram_range=(1,2))

X_train_bigram_tfidf = bigram_tfidf_vect.fit_transform(train['text'])

bigram_tfidf_vector = bigram_tfidf_vect.transform([sentence]).toarray()
indices = np.where(np.any(bigram_tfidf_vector!=0, axis=0))[0]
index_to_word = {index : word for word, index in bigram_tfidf_vect.vocabulary_.items()}

print(f"The size of the train dataset is {X_train_bigram_tfidf.shape}")
print(X_train_bigram_tfidf.count_nonzero)
print(sentence)

[(ind, index_to_word[ind], bigram_tfidf_vector[0, ind]) for ind in indices]

The size of the train dataset is (17621, 230440)
<bound method _data_matrix.count_nonzero of <17621x230440 sparse matrix of type '<class 'numpy.float64'>'
	with 813907 stored elements in Compressed Sparse Row format>>
The gigantic magnitude and the immediately available nature of the sum, dazzled and bewildered all who thought upon the topic.


[(4324, 'all', 0.088547269675905),
 (4855, 'all who', 0.2335206648733459),
 (7667, 'and', 0.0829985041349335),
 (8116, 'and bewildered', 0.24831875690585964),
 (12202, 'and the', 0.09948207535251821),
 (19515, 'available', 0.26664786053577216),
 (25467, 'bewildered', 0.21166054964603465),
 (45745, 'dazzled', 0.24831875690585964),
 (75382, 'gigantic', 0.18316632948172856),
 (93018, 'immediately', 0.16068071922477745),
 (112540, 'magnitude', 0.22156868352708564),
 (112541, 'magnitude and', 0.24831875690585964),
 (124876, 'nature', 0.13929051748936838),
 (124942, 'nature of', 0.1894923535718752),
 (131308, 'of', 0.040640907290130535),
 (134836, 'of the', 0.06654221632472784),
 (181040, 'sum', 0.20446972294810256),
 (186670, 'the', 0.13856949392524126),
 (189413, 'the gigantic', 0.24831875690585964),
 (192991, 'the sum', 0.2335206648733459),
 (193291, 'the topic', 0.23759691861052884),
 (199054, 'thought', 0.12714459829609193),
 (199173, 'thought upon', 0.2559260222404413),
 (203935, 'topi

Using bigrams (trigrams) will probably greatly improve the performance of the solution based on tf-idf embeddings.  

#### Advantages and disadvantages

$"+"$:
1. Outperforms 'bag of words' approach and solves some of its problems

$"-"$:
1. Still doesn't capture meaning

Generally, Tf-Idf can be used as a baseline technique for producing text embeddings and text classification (for example, tf-idf + LogReg with hyperparameter optimization).


#### Baseline model

To evaluate embeddings we will stick to extrinsic evaluation - evaluation on real task - text classification

In [37]:
tfidf_vect_eval = TfidfVectorizer(ngram_range=(1,1))
X_train_tfidf = tfidf_vect_eval.fit_transform(train['text'])
X_val_tfidf = tfidf_vect_eval.transform(val['text'])

In [38]:
logreg_tfidf = LogisticRegression(solver='sag', max_iter=10000)
logreg_tfidf.fit(X_train_tfidf, train['author'])

LogisticRegression(max_iter=10000, solver='sag')

In [39]:
pred = logreg_tfidf.predict(X_val_tfidf)
f1 = f1_score(pred, val['author'], average='macro')
f1

0.816697787096243

### Distributional Semantics

Words which frequently appear in similar contexts have similar meaning.

Often you can find it formulated as "You shall know a word by the company it keeps" with the reference to J. R. Firth in 1957, but actually there were a lot more people responsible, and much earlier. For example, Harris, 1954.

This is an extremely valuable idea: it can be used in practice to make word vectors capture their meaning. According to the distributional hypothesis, "to capture meaning" and "to capture contexts" are inherently the same. Therefore, all we need to do is to put information about word contexts into word representation.

Main idea: We need to put information about word contexts into word representation.

[Distributional Semantics from Lena Voita's NLP Course | For You](https://lena-voita.github.io/nlp_course/word_embeddings.html#distributional_semantics)

### Count-Based Methods

How: Put this information manually based on global corpus statistics.
<img src="https://lena-voita.github.io/resources/lectures/word_emb/preneural/idea-min.png" width="800">

The procedure consists out of three steps:
1. Construct a word-context matrix
2. Reduce its dimensionality
    * a raw matrix is very large
    * a lot of words appear in only a few of possible contexts, this matrix potentially has a lot of uninformative elements (e.g., zeros)
3. Normalize word vectors (to prepare them for similarity estimation)

To define a count-based method, we need to define two things:
* possible contexts (including what does it mean that a word appears in a context),
* the notion of association, i.e., formulas for computing matrix elements.

[Count-Based Methods from Lena Voita's NLP Course | For You](https://lena-voita.github.io/nlp_course/word_embeddings.html#pre_neural)

#### Co-Occurence Counts

##### Theory

* *Context*: surrounding words in a L-sized window
* *Matrix element*: N(w, c) - number of time the *word **w*** appears in *context **c***  

In this implementation columns will represent context words and index will represent center words. Co-occurrence matrix will be square-shaped.

##### Practice

Let's tokenize our `train` and `val` corpora

In [40]:
tokenized_train = [tokenizer(text.lower()) for text in train['text']]
tokenized_val = [tokenizer(text.lower()) for text in val['text']]

Unique words are needed to create co-occurence matrix.  
I will be using dataframe to wrap matrix for better representation, although loosing some speed

In [41]:
unique_words = sorted(list(set(chain.from_iterable(tokenized_train))))
co_occurrence_dataframe = pd.DataFrame(0, columns=unique_words, index=unique_words)

Example: first tokenized sentence

In [42]:
tokenized_train[0]

['still',
 'others',
 'including',
 'joe',
 'himself',
 'have',
 'theories',
 'too',
 'wild',
 'and',
 'fantastic',
 'for',
 'sober',
 'credence']

Co-occurrence matrix is initialized with zeros

In [43]:
co_occurrence_dataframe.loc['still', 'others']

0

In [44]:
def get_window(text: List[str], window_size: int) -> Iterator[Tuple[str, List[str]]]:
    for backward, current in enumerate(range(len(text)), start=0 - (window_size // 2)):
        if backward < 0:
            backward = 0
        context = list(text[backward:current]) + list(text[current + 1:current + 1 + window_size // 2])
        center = text[current]
        yield center, context

This function will generate center words and their contexts for every sentence given the tokenized sentence and the size of the window.

In [45]:
for center, context_values in get_window(tokenized_train[0], 5):
    print(center, context_values)

still ['others', 'including']
others ['still', 'including', 'joe']
including ['still', 'others', 'joe', 'himself']
joe ['others', 'including', 'himself', 'have']
himself ['including', 'joe', 'have', 'theories']
have ['joe', 'himself', 'theories', 'too']
theories ['himself', 'have', 'too', 'wild']
too ['have', 'theories', 'wild', 'and']
wild ['theories', 'too', 'and', 'fantastic']
and ['too', 'wild', 'fantastic', 'for']
fantastic ['wild', 'and', 'for', 'sober']
for ['and', 'fantastic', 'sober', 'credence']
sober ['fantastic', 'for', 'credence']
credence ['for', 'sober']


**Step 1**: *Construct a word-context matrix*

In [46]:
window_size = 5
for tokenized_sentence in tqdm(tokenized_train):
    for center, context_values in get_window(tokenized_sentence, window_size):
        for context_word in context_values:
            co_occurrence_dataframe.at[center, context_word] += 1

  0%|          | 0/17621 [00:00<?, ?it/s]

In [47]:
co_occurrence_dataframe.loc['still', 'others']

2

2 means that the word 'still' is center word for word 'others' in two sentences.

**Step 2**: *Reduce word-context matrix dimensionality*

In [48]:
X = csr_matrix(co_occurrence_dataframe)
svd = TruncatedSVD(n_components=1000, n_iter=7, random_state=SEED)
svd.fit(X)

TruncatedSVD(n_components=1000, n_iter=7, random_state=42)

We'll be downsizing word vectors using truncated SVD down to the size of 1000 elements.

In [49]:
word_vectors = svd.transform(X)
word_vectors.shape

(24069, 1000)

**Step 3**: *Normalize word vectors*

In [50]:
norm(word_vectors, axis=1)

array([1.09423007, 3.03860193, 1.4305215 , ..., 2.23992585, 2.44889925,
       1.00181852])

In [51]:
word_vectors /= norm(word_vectors, axis=1).reshape(-1,1)

In [52]:
norm(word_vectors, axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

##### Baseline model

To evaluate embeddings we will stick to extrinsic evaluation - evaluation on real task - text classification

In [53]:
embeddings_lookup_co_occurrence = dict(zip(co_occurrence_dataframe.index.values, word_vectors))

Dictionary `embeddings_lookup_co_occurrence` will return vector for the every word in our training set

In [54]:
def sent2vec(tokenized_sentence, embeddings_lookup):
    word_embeddings = []
    for word in tokenized_sentence:
        try:
            word_embeddings.append(embeddings_lookup[word])
        except:
            continue
    word_embeddings = np.array(word_embeddings)
    v = word_embeddings.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(100)
    return v / norm(v, axis=0)

In order to get the embedding for the sentence we will sum word embeddings along every dimension and then normalize it 

In [55]:
X_train_cooc_counts = [sent2vec(tokenized_sentence, embeddings_lookup_co_occurrence) for tokenized_sentence in tqdm(tokenized_train)]
X_val_cooc_counts = [sent2vec(tokenized_sentence, embeddings_lookup_co_occurrence) for tokenized_sentence in tqdm(tokenized_val)]

  0%|          | 0/17621 [00:00<?, ?it/s]

  0%|          | 0/1958 [00:00<?, ?it/s]

In [56]:
logreg_cooc_counts = LogisticRegression(solver='sag', max_iter=10000)
logreg_cooc_counts.fit(X_train_cooc_counts, train['author'])

LogisticRegression(max_iter=10000, solver='sag')

In [57]:
pred = logreg_cooc_counts.predict(X_val_cooc_counts)
f1_cooc = f1_score(pred, val['author'], average='macro')
print(f"F1 score for simple co-occurence counts implementation is {f1_cooc.round(2)}")

F1 score for simple co-occurence counts implementation is 0.66


The performance is even worse than the performance of 'bag of words' + 'LogReg' model

#### Positive Pointwise Mutual Information (PPMI)

##### Theory

* *Context*: surrounding words in a L-sized window
* *Matrix element*: $PPMI(w, c) = max(0, PMI(w, c))$

In this implementation columns will represent context words and index will represent center words. Co-occurrence matrix will be square-shaped.

##### Practice

In [61]:
ppmi_dataframe = co_occurrence_dataframe.copy()

We'll be using co-occurrence matrix computed on previous step in new computations

In [62]:
alpha = 0.75
window_size = 5

sum_cooccurrences = co_occurrence_dataframe.to_numpy().sum()
row_counts = co_occurrence_dataframe.sum(axis=0).to_dict()
column_counts = co_occurrence_dataframe.sum(axis=1).to_dict()
column_counts_squared = (np.array(list(column_counts.values())) ** alpha).sum()

**Step 1**: *Construct a word-context matrix*

In [63]:
for tokenized_sentence in tqdm(tokenized_train):
    for center, context_values in get_window(tokenized_sentence, window_size):
        for context_word in context_values:
            p_ij = co_occurrence_dataframe.at[center, context_word] / sum_cooccurrences
            p_i = row_counts[center] / sum_cooccurrences
            # p_j = column_counts[context_word] ** alpha / column_counts_squared
            p_j = column_counts[context_word] / sum_cooccurrences
            ppmi_dataframe.at[center, context_word] = max(0, np.log2(p_ij / (p_i * p_j)) if p_ij > 0 else 0)

  0%|          | 0/17621 [00:00<?, ?it/s]

In [64]:
ppmi_dataframe.loc['still', 'others']

2

**Step 2**: *Reduce its dimensionality*

In [65]:
X = csr_matrix(ppmi_dataframe)
svd = TruncatedSVD(n_components=1000, n_iter=7, random_state=SEED)
svd.fit(X)

TruncatedSVD(n_components=1000, n_iter=7, random_state=42)

In [66]:
word_vectors = svd.transform(X)
word_vectors.shape

(24069, 1000)

**Step 3**: *Normalize word vectors*

In [67]:
norm(word_vectors, axis=1)

array([ 5.43257234, 10.49117444,  5.74282706, ...,  4.01048542,
        4.81776306,  3.29906746])

In [68]:
word_vectors /= norm(word_vectors, axis=1).reshape(-1,1)

In [69]:
norm(word_vectors, axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

##### Baseline model

To evaluate embeddings we will stick to extrinsic evaluation - evaluation on real task - text classification

In [70]:
embeddings_lookup = dict(zip(ppmi_dataframe.index.values, word_vectors))

In [71]:
X_train_cooc_ppmi = [sent2vec(tokenized_sentence, embeddings_lookup) for tokenized_sentence in tqdm(tokenized_train)]
X_val_cooc_ppmi = [sent2vec(tokenized_sentence, embeddings_lookup) for tokenized_sentence in tqdm(tokenized_val)]

  0%|          | 0/17621 [00:00<?, ?it/s]

  0%|          | 0/1958 [00:00<?, ?it/s]

In [72]:
logreg_cooc_ppmi = LogisticRegression(solver='sag', max_iter=10000)
logreg_cooc_ppmi.fit(X_train_cooc_ppmi, train['author'])

LogisticRegression(max_iter=10000, solver='sag')

In [73]:
pred = logreg_cooc_ppmi.predict(X_val_cooc_ppmi)
f1_ppmi = f1_score(pred, val['author'], average='macro')
print(f"F1 score for PPMI implementation is {f1_ppmi.round(2)}")

F1 score for PPMI implementation is 0.8


#### Latent Semantic Analysis (LSA): Understanding Documents

For this approach I will be using *20 newsgroups dataset*

##### Loading the dataset

In [3]:
data_all_train = fetch_20newsgroups(
    shuffle=True,
    random_state=SEED,
    remove=('headers', "footers", "quotes"),
    return_X_y=False,
    subset='all'
)

In [4]:
target_names = data_all_train['target_names']
labels = data_all_train['target']
data = data_all_train['data']

categories_map = {i:category for i, category in enumerate(target_names)}
named_labels = [categories_map[label] for label in labels]

In [5]:
news_df = pd.DataFrame({"text":data, "labels":named_labels})
news_df.head()

,text,labels
0,\n\nI am sure some bashers of Pens fans are pr...,rec.sport.hockey
1,My brother is in the market for a high-perform...,comp.sys.ibm.pc.hardware
2,\n\n\n\n\tFinally you said what you dream abou...,talk.politics.mideast
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,comp.sys.ibm.pc.hardware
4,1) I have an old Jasmine drive which I cann...,comp.sys.mac.hardware


In [13]:
tfidf_vect_news = TfidfVectorizer()
X_train_news = tfidf_vect_news.fit_transform(news_df['text'])
tokenizer_tf_idf = tfidf_vect_news.build_tokenizer()

In [18]:
tokenizer_news = [tokenizer_tf_idf(text.lower()) for text in news_df['text']]

In [19]:
unique_words = sorted(list(set(chain.from_iterable(tokenizer_news))))
unique_docs = ["doc_" + str(i) for i in news_df.index]

**Step 1**: *Construct a word-context matrix*

In [75]:
lsa_dataframe = pd.DataFrame.sparse.from_spmatrix(X_train_news, columns=unique_words, index=unique_docs)

**Step 2**: *Reduce its dimensionality*

In [76]:
X = csr_matrix(lsa_dataframe)
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=SEED)
svd.fit(X)

TruncatedSVD(n_components=100, n_iter=7, random_state=42)

In [77]:
len(svd.components_)

100

In [78]:
len(svd.components_[0])

134410

In [79]:
document_vectors = svd.transform(X)
document_vectors.shape

(18846, 100)

**Step 3**: *Normalize document vectors*

In [80]:
document_vectors /= norm(document_vectors, axis=1).reshape(-1,1)

C:\Users\ryblo\AppData\Local\Temp/ipykernel_6796/1196041914.py:1: RuntimeWarning: invalid value encountered in true_divide
  document_vectors /= norm(document_vectors, axis=1).reshape(-1,1)


In [96]:
documents_lookup = dict(zip(lsa_dataframe.index.values, document_vectors))

In [97]:
documents_to_topic_lookup = dict(zip(lsa_dataframe.index.values, named_labels))
topic_to_documents_lookup = {}
for k, v in documents_to_topic_lookup.items():
    topic_to_documents_lookup[v] = topic_to_documents_lookup.get(v, []) + [k]

In [98]:
target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [99]:
topic_to_documents_lookup['alt.atheism'][:5]

['doc_14', 'doc_29', 'doc_56', 'doc_140', 'doc_173']

In [100]:
topic_to_documents_lookup['sci.electronics'][:5]

['doc_5', 'doc_16', 'doc_17', 'doc_42', 'doc_50']

In [110]:
cosine_similarity(documents_lookup['doc_14'].reshape(1, -1), documents_lookup['doc_42'].reshape(1, -1))

array([[0.19004197]])

In [114]:
import itertools
electronics_combinations = list(itertools.combinations(topic_to_documents_lookup['sci.electronics'], 2))
atheism_combinations = list(itertools.combinations(topic_to_documents_lookup['alt.atheism'], 2))

In [152]:
el_similarity = 0
for el_comb in tqdm(electronics_combinations):
    try:
        el_similarity += cosine_similarity(documents_lookup[el_comb[0]].reshape(1, -1), documents_lookup[el_comb[1]].reshape(1, -1))
    except:
        el_similarity += 0
el_similarity /= len(electronics_combinations)
el_similarity

  0%|          | 0/483636 [00:00<?, ?it/s]

array([[0.30986899]])

In [151]:
at_similarity = 0
for el_comb in tqdm(atheism_combinations):
    try:
        at_similarity += cosine_similarity(documents_lookup[el_comb[0]].reshape(1, -1), documents_lookup[el_comb[1]].reshape(1, -1))
    except:
        at_similarity += 0
at_similarity /= len(atheism_combinations)
at_similarity

  0%|          | 0/318801 [00:00<?, ?it/s]

array([[0.35138187]])

In [135]:
all_combinations = [list(zip(each_permutation, topic_to_documents_lookup['alt.atheism'][:1000])) for each_permutation in itertools.combinations(topic_to_documents_lookup['sci.electronics'][:1000], len(topic_to_documents_lookup['sci.electronics'][:1000]))]

In [150]:
inter_similarity = 0
for el_comb in tqdm(all_combinations[0]):
    try:
        sim = cosine_similarity(documents_lookup[el_comb[0]].reshape(1, -1), documents_lookup[el_comb[1]].reshape(1, -1))
    except:
        sim = 0
    inter_similarity += sim

inter_similarity /= len(all_combinations[0])
inter_similarity

  0%|          | 0/799 [00:00<?, ?it/s]

array([[0.30232196]])

### TODO: 
* Finish item 3.1.4 about using stopwords 
* Add the reason to use logarithms for tf-idf (sublinear tf, etc.) [1](https://stats.stackexchange.com/questions/161640/understanding-the-use-of-logarithms-in-the-tf-idf-logarithm) and [2](https://stackoverflow.com/questions/27067992/why-is-log-used-when-calculating-term-frequency-weight-and-idf-inverse-document)
* Write down questions
    * bag of words
    * tf-idf
* Fill items 3.3-3.4 with information and refactor the code
* Elaborate the formula of PPMI using [3](https://web.stanford.edu/~jurafsky/li15/lec3.vector.pdf)
    

In [4]:
np.log2(1_000_000)

19.931568569324174

In [ ]:
np.log2(2_000_000)